In [1]:
import os
for dirname, _, filenames in os.walk('ParticipantData_BTPC/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

ParticipantData_BTPC/.DS_Store
ParticipantData_BTPC/Sample Submission.csv
ParticipantData_BTPC/Test.csv
ParticipantData_BTPC/submission.csv
ParticipantData_BTPC/Train.csv


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [4]:
train=pd.read_csv("ParticipantData_BTPC/Train.csv")
train.head()

,session_id,session_number,client_agent,device_details,date,purchased,added_in_cart,checked_out,time_spent
0,57f879e70d3c5fc2a98102d64c9fd84e,715,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Desktop - Chrome,2020-01-22,1,0,0,236.886
1,a5442b0c7c33d0a811e7661e556b2de8,55,Product/8.0 iPhone/8.1.3,iPhone - iOS,2020-02-27,1,0,0,1277.455
2,305cb1486ed8610c00b37007926cb2c4,11,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,iPhone - MobileWeb,2019-08-01,0,0,0,21.704
3,f2c1ecc9993f0071df91ba178450498c,2794,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Desktop - IE,2019-12-30,0,1,0,455.201
4,e460830ae295e55d2216ebdc761ab9a6,3674,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like ...,iPhone - Web,2019-09-10,0,0,0,1164.877


In [5]:
train=train.drop(["session_id","client_agent"],axis=1)

Splitting date into yy-mm-dd columns

In [6]:
yy=[]
mm=[]
dd=[]

for each in train.date.values:
  string_rp=each.split('-')
  yy.append(int(string_rp[0]))
  mm.append(int(string_rp[1]))
  dd.append(int(string_rp[2]))

train["yy"]=pd.DataFrame(yy)
train["mm"]=pd.DataFrame(mm)
train["dd"]=pd.DataFrame(dd)

In [7]:
train=train.drop(["date"],axis=1)
train.head()

,session_number,device_details,purchased,added_in_cart,checked_out,time_spent,yy,mm,dd
0,715,Desktop - Chrome,1,0,0,236.886,2020,1,22
1,55,iPhone - iOS,1,0,0,1277.455,2020,2,27
2,11,iPhone - MobileWeb,0,0,0,21.704,2019,8,1
3,2794,Desktop - IE,0,1,0,455.201,2019,12,30
4,3674,iPhone - Web,0,0,0,1164.877,2019,9,10


Extracting target labels from dataset

In [8]:
train_y=train["time_spent"].values
train=train.drop(["time_spent"],axis=1)

Applying Label Encoder on columns to encode labels

In [9]:
for each in ["device_details"]:
  train[each]=LabelEncoder().fit_transform(train[each])

In [10]:
train.head()

,session_number,device_details,purchased,added_in_cart,checked_out,yy,mm,dd
0,715,6,1,0,0,2020,1,22
1,55,16,1,0,0,2020,2,27
2,11,14,0,0,0,2019,8,1
3,2794,8,0,1,0,2019,12,30
4,3674,15,0,0,0,2019,9,10


Performing scaling on train dataset

In [11]:
from sklearn.preprocessing import RobustScaler
train=RobustScaler().fit_transform(train)

Using Random Forest Regressor for training the model

In [16]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='squared_error',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
model.fit(train,train_y)

RandomForestRegressor(n_estimators=1000)

In [17]:
test=pd.read_csv("ParticipantData_BTPC/Test.csv")
test=test.drop(["client_agent","session_id"],axis=1)

In [18]:
yy=[]
mm=[]
dd=[]

for each in test.date.values:
  string_rp=each.split('-')
  yy.append(int(string_rp[0]))
  mm.append(int(string_rp[1]))
  dd.append(int(string_rp[2]))

test["yy"]=pd.DataFrame(yy)
test["mm"]=pd.DataFrame(mm)
test["dd"]=pd.DataFrame(dd)

test=test.drop(["date"],axis=1)
test.head(5)

,session_number,device_details,purchased,added_in_cart,checked_out,yy,mm,dd
0,99,Desktop - IE,1,0,0,2020,1,7
1,935,Android Phone - Android,0,0,0,2020,2,13
2,902,iPhone - iOS,1,0,0,2019,10,14
3,3223,Desktop - IE,0,0,0,2020,1,26
4,231,Desktop - Chrome,0,0,0,2020,1,29


In [19]:
for each in ["device_details"]:
  test[each]=LabelEncoder().fit_transform(test[each])
test.head(5)

,session_number,device_details,purchased,added_in_cart,checked_out,yy,mm,dd
0,99,7,1,0,0,2020,1,7
1,935,0,0,0,0,2020,2,13
2,902,15,1,0,0,2019,10,14
3,3223,7,0,0,0,2020,1,26
4,231,5,0,0,0,2020,1,29


In [20]:
test=RobustScaler().fit_transform(test)

In [21]:
model.predict(test)

array([1102.58181567,  169.209323  , 1166.978557  , ...,  128.947988  ,
       1268.178274  ,  947.509997  ])

In [22]:
submit=pd.read_csv("ParticipantData_BTPC/Sample Submission.csv")

In [23]:
submit["time_spent"]=pd.DataFrame(model.predict(test))

In [24]:
submit.to_csv("ParticipantData_BTPC/Submit.csv",index=False)